<a href="https://colab.research.google.com/github/AiProcess/CartPole_RL/blob/main/cart_pole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchrl
!pip install tensordict

In [2]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Default device is {device}")

Default device is cpu


# Environment

In [3]:
from torchrl.envs import GymEnv
from torchrl.envs.transforms import StepCounter, TransformedEnv
from torchrl.record import VideoRecorder, CSVLogger

## Main Env

In [4]:
env = TransformedEnv(
    env=GymEnv(env_name="CartPole-v1"),
    transform=StepCounter(),
).to(device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## Test Env

In [5]:
csv_logger = CSVLogger(exp_name="test", log_dir="cart_pole", video_format="mp4")
video_recorder = VideoRecorder(logger=csv_logger, tag="video")

test_env = TransformedEnv(
    env=GymEnv(
        env_name="CartPole-v1",
        from_pixels=True,
        pixels_only=False
    ),
    transform=video_recorder
).to(device)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.10/dist-packages/pkg_resources/__init__.py:3154: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('google')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  declare_namespace(pkg)
/usr/local/

In [6]:
# env.rollout(max_steps=5)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Agent

In [7]:
from torch import nn
import torch.nn.functional as F
from tensordict.nn import TensorDictModule, TensorDictSequential
from torchrl.modules import EGreedyModule, QValueModule

## Policy

In [8]:
class MLP_Model(nn.Module):
    def __init__(self, in_features, out_features) -> None:
        super().__init__()
        self.layer1 = nn.Linear(in_features=in_features, out_features=64)
        self.layer2 = nn.Linear(in_features=64, out_features=64)
        self.layer3 = nn.Linear(in_features=64, out_features=out_features)

    def forward(self, x):
        y = F.relu(self.layer1(x))
        y = F.relu(self.layer2(y))
        y = F.relu(self.layer3(y))

        return y

In [9]:
value_mlp = MLP_Model(
    in_features=env.observation_spec['observation'].shape[0],
    out_features=env.action_spec.shape[0]
)

value_net = TensorDictModule(
    module=value_mlp,
    in_keys=["observation"],
    out_keys=["action_value"]
)

exploration_module = EGreedyModule(
    env.action_spec, annealing_num_steps=100_000, eps_init=0.5
)

policy_explore = TensorDictSequential(
    value_net,
    QValueModule(spec=env.action_spec),
    exploration_module
).to(device)

## Data Collection

# Training Loop

# Results